In [2]:
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

import matplotlib.colors as mc
import math
import matplotlib.pyplot as plt

In [38]:
from PIL import Image, ImageDraw
import argparse
import sys
import webcolors

from math import sqrt

COLORS = (
    (255,255,255),
    (255,0,0),
    (0,255,0),
    (0,0,255),
    (255,255,0),
    (0,255,255),
    (255,0,255),
    (192,192,192),
    (128,128,128),
    (128,0,0),
    (128,128,0),
    (0,128,0),
    (128,0,128),
    (0,128,128),
    (0,0,128)
)

def closest_color(rgb):
    r, g, b = rgb
    color_diffs = []
    for color in COLORS:
        cr, cg, cb = color
        color_diff = sqrt(abs(r - cr)**2 + abs(g - cg)**2 + abs(b - cb)**2)
        color_diffs.append((color_diff, color))
    return min(color_diffs)[1]

def get_colors(image_file, numcolors=10, resize=150):
    # Resize image to speed up processing
    img = Image.open(image_file)
    img = img.copy()
    img.thumbnail((resize, resize))

    # Reduce to palette
    paletted = img.convert('P', palette=Image.ADAPTIVE, colors=numcolors)

    # Find dominant colors
    palette = paletted.getpalette()
    color_counts = sorted(paletted.getcolors(), reverse=True)
    colors = list()
    for i in range(numcolors):
        palette_index = color_counts[i][1]
        dominant_color = palette[palette_index*3:palette_index*3+3]
        colors.append(tuple(dominant_color))

    return colors

def get_mask_image(mask, box):
    mask_h = int(math.ceil(box[3] - box[1]))
    mask_w = int(math.ceil(box[2] - box[0]))

    original_img = np.array(Image.fromarray(cv2.imread("/home/lkunam/Downloads/test.jpg")))

    temp_mask = np.zeros((mask_h, mask_w))

    for h_idx in range(int(box[1]), int(box[3])):
        for w_idx in range(int(box[0]), int(box[2])):
            temp_mask[h_idx - int(box[1])][w_idx - int(box[0])] = mask[h_idx][w_idx]

    temp_masks_ints = temp_mask.astype(int)

    temp_mask_fill = np.zeros((mask_h, mask_w, 4))
    for h_idx, h_bw in enumerate(temp_masks_ints):
        for w_idx, w_bw in enumerate(h_bw):
            if (w_bw == 0):
                temp_mask_fill[h_idx][w_idx] = [0,0,0,0]
            else:
                orig_w = int(math.ceil(w_idx + box[0]))
                orig_h = int(math.ceil(w_idx + box[1]))
                temp_mask_fill[h_idx][w_idx] = np.append(original_img[orig_h][orig_w],1)

    return temp_mask_fill

#cv2.imshow()

In [4]:
image = "/home/lkunam/Downloads/test.jpg"
new_color = get_colors(image,1)
print(new_color[0])
named_color = closest_color(new_color[0])
print(named_color)

(144, 107, 103)
(99, 23, 153)


In [5]:
def get_color_from_masks(image):
    detectron2_img = cv2.imread(image)
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
    # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    predictor = DefaultPredictor(cfg)
    outputs = predictor(detectron2_img)

    masks = outputs["instances"].pred_masks.cpu().numpy()
    boxes = outputs["instances"].pred_boxes.tensor.cpu().numpy()
    
    masks_arr = []
    
    for index, mask in enumerate(masks):
        masks_arr.append(get_mask_image(mask, boxes[index], detectron2_img))
    

In [32]:
detectron2_img = cv2.imread("/home/lkunam/Downloads/test.jpg")
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(detectron2_img)
mask = outputs["instances"].pred_masks.cpu().numpy()[0]
box = outputs["instances"].pred_boxes.tensor.cpu().numpy()[0]

mask_h = int(math.ceil(box[3] - box[1]))
mask_w = int(math.ceil(box[2] - box[0]))

original_img = np.array(Image.fromarray(cv2.imread("/home/lkunam/Downloads/test.jpg")))

temp_mask = np.zeros((mask_h, mask_w))

for h_idx in range(int(box[1]), int(box[3])):
    for w_idx in range(int(box[0]), int(box[2])):
        temp_mask[h_idx - int(box[1])][w_idx - int(box[0])] = mask[h_idx][w_idx]

temp_masks_ints = temp_mask.astype(int)

temp_mask_fill = np.zeros((mask_h, mask_w, 4))
for h_idx, h_bw in enumerate(temp_masks_ints):
    for w_idx, w_bw in enumerate(h_bw):
        if (w_bw == 0):
            temp_mask_fill[h_idx][w_idx] = [0,0,0,0]
        else:
            orig_w = int(math.ceil(w_idx + box[0]))
            orig_h = int(math.ceil(w_idx + box[1]))
            temp_mask_fill[h_idx][w_idx] = np.append(original_img[orig_h][orig_w],1)


In [33]:
cv2.imwrite("test_image.png",temp_mask_fill[:,:,0:3])
random = Image.open("test_image.png")
new_color = get_colors("test_image.png",1)

In [36]:
new_color = get_colors("test_image.png",1)

In [39]:
closest_color(new_color[0])

(128, 128, 128)

In [23]:
new_color
os.remove("test_image.png")

In [81]:
test = get_mask_image(masks, boxes, image)

In [1]:
temp_mask_fill

NameError: name 'temp_mask_fill' is not defined